In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib

Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_recreate_milvus.ipynb.
Converted 00_tools.ipynb.
Converted 02_onehot.ipynb.
Converted 03_quantization.ipynb.
Converted 04_milvus.ipynb.
Converted 06_clip.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted milvus_dev.ipynb.
